In [53]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import random
import math

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

BUILD_INDEX = True
BUILD_INDEX_TEST = False
TEST_QUERY = False
pd.set_option('display.max_colwidth', None)
virgin_index_creation = True

###function to create inverted index

In [54]:
if BUILD_INDEX: 

    def inverted_indexCreation(inputFile):
#         stop =5 # for debugging
        new_word = True
        wordsAdded = {}

        data_type = {'Document_Id' : 'int',
                     'Profile_Name':'str',
                     'Title':'str',
                     'Co_Authors':'str',
                     'Title_Link': 'str',
                     'No_Of_Citations':'str'
                    }

        data = pd.read_csv(inputFile,header='infer',dtype=data_type)

        exlcude_puncuation = [';', '.', ',', '!', ':', '-','_']
        #set stopwords for english language
        sw = stopwords.words('english')

        ps=PorterStemmer()
        filtered_docs = []
        docs = data['Profile_Name']+" "+data['Title']+" "+data['Co_Authors'].astype('str')# combine name title year for indexing
#         print(docs)
        for title,doc_id in zip(docs,data.Document_Id):
            tokens = word_tokenize(str(title))
            temp = ''
            word_count =1
#             print("\n",tokens,"WC",word_count)
#             break
            for word in tokens:
                if word not in sw and word not in exlcude_puncuation and len(word)>2:
                    word_stem =ps.stem(word) # stem the word
                    if word_stem not in wordsAdded.keys(): # Create a new entry in dictionary for new words : terms and posting
                        wordsAdded[word_stem]=[]
                        word_count=1
                        new_word =True
                    else:
                        new_word=False


                    if new_word==True: # Adding word and count first time into dicttionary
                        #print("inside ..",word_stem,doc_id,word_count,"\n")
                        wordsAdded[word_stem].append((doc_id,word_count))
                        word_count +=1 # increment the existing word count
                    else: # only update the doc_id and count for word                        
                        #find position of tuple to update count of word in the index dictionary
                        val_length = len(wordsAdded[word_stem]) # find how many entries are already made for word
#                         print("\nAlready existing dict : ", word_stem, wordsAdded[word_stem])
                        for count in range(val_length):
                            if wordsAdded[word_stem][count][0] == doc_id : # find the location of tuple with matching doc id 
                                location=True
                                position=count
                                break # found the location of tuple for matching doc_id so break the loop

                        if location == True:
                            existing_word_count = wordsAdded[word_stem][position][1]
                            wordsAdded[word_stem][position] = (doc_id, existing_word_count+1)
                            #reset flags
                            location = False
                            position=-1
                        else:  # its a new doc_id entry for existing word
                            # add a new doc_id and count info
                            wordsAdded[word_stem].append((doc_id, 1))

#                         print("\nAlready existing dict (After update): ", word_stem, wordsAdded[word_stem])
                        #print("inside ..",word_stem,doc_id+1,word_count,"\n")
                    temp = temp + word_stem + " "

            filtered_docs.append(temp)
#             stop=stop-1
#             if stop <0: break
        data['Filtered_title'] =  filtered_docs 

        return wordsAdded,data
        
        

In [55]:
# Update already created index
# return the filtered new data and update the global inverted index
#input dataframe with channged rows
def update_index(data):
    global inv_index
    new_word = True
    
    exlcude_puncuation = [';', '.', ',', '!', ':', '-','_']
    #set stopwords for english language
    sw = stopwords.words('english')
    ps=PorterStemmer()
    filtered_docs = []    
    
    docs = data['Profile_Name']+" "+data['Title']+" "+data['Co_Authors'].astype('str')# combine name title year for indexing 
    
    if virgin_index_creation == False:
        if len(inv_index):
            for title,doc_id in zip(docs,data.Document_Id):
                tokens = word_tokenize(str(title))
                temp = ''
                word_count =1
    #             print("\n",tokens,"WC",word_count)
    #             break
                for word in tokens:
                    if word not in sw and word not in exlcude_puncuation and len(word)>2:
                        word_stem =ps.stem(word) # stem the word
                        if word_stem not in inv_index.keys(): # Create a new entry in dictionary for new words : terms and posting
                            inv_index[word_stem]=[]
                            word_count=1
                            new_word =True
                        else:
                            new_word=False


                        if new_word==True: # Adding word and count first time into dicttionary
                            #print("inside ..",word_stem,doc_id,word_count,"\n")
                            inv_index[word_stem].append((doc_id,word_count))
                            word_count +=1 # increment the existing word count
                        else: # only update the doc_id and count for word                        
                            #find position of tuple to update count of word in the index dictionary
                            val_length = len(inv_index[word_stem]) # find how many entries are already made for word
    #                         print("\nAlready existing dict : ", word_stem, wordsAdded[word_stem])
                            for count in range(val_length):
                                if inv_index[word_stem][count][0] == doc_id : # find the location of tuple with matching doc id 
                                    location=True
                                    position=count
                                    break # found the location of tuple for matching doc_id so break the loop

                            if location == True:
                                existing_word_count = inv_index[word_stem][position][1]
                                inv_index[word_stem][position] = (doc_id, existing_word_count+1)
                                #reset flags
                                location = False
                                position=-1
                            else:  # its a new doc_id entry for existing word
                                # add a new doc_id and count info
                                inv_index[word_stem].append((doc_id, 1))

    #                         print("\nAlready existing dict (After update): ", word_stem, wordsAdded[word_stem])
                            #print("inside ..",word_stem,doc_id+1,word_count,"\n")
                        temp = temp + word_stem + " "

                filtered_docs.append(temp)
    #             stop=stop-1
    #             if stop <0: break
                data['Filtered_title'] =  filtered_docs 
            print("Update Completed Successfully...")
            return data           
         
        else:
            print("Inverted index is empty..please first create the index")            
    else:
        print("Please first create inverted index..")
        
    

In [56]:
if BUILD_INDEX:
    if virgin_index_creation:  # create whole index for the first time
        articles_inputFile = "Scraped_Articles_Details.csv"
        start_time = time.time() # measure time taken to build index 

        display_data = pd.read_csv(articles_inputFile,header='infer')
        display_data.fillna(value=0,inplace=True)
        inv_index, data =inverted_indexCreation(articles_inputFile)

        stop_time = time.time() # stop the timer
        time_taken = stop_time-start_time # time taken to build index
        print("\nTime taken to build index :",str(np.round(time_taken,4))+" sec") 
        print("Index Size", len(inv_index))
        virgin_index_creation = False
        boot_file = open("robots.txt",'w')
        boot_file.write("0")
        boot_file.close()
    


Time taken to build index : 2.4963 sec
Index Size 5082


In [57]:
#Index update main function
def index_update():
    read_file = open("robots.txt",'r')
    flag = read_file.read()
    if int(flag) ==0:
        print("Index update only")

        # Compare the data of last crawl with latest crawl data
        last_file = "Scraped_Articles_Details_Last.csv"
        latest_file = "Scraped_Articles_Details.csv"

        last_data = pd.read_csv(last_file,header='infer')
        latest_data = pd.read_csv(latest_file, header='infer')

        #check number of rows in each of the data
        size_last = len(last_data)
        size_latest= len(latest_data)

        print("Old File size : ",size_last)
        print("New File size : ",size_latest)
        
        if size_last==size_latest:
            difference = latest_data[last_data.ne(latest_data).any(axis=1)]

        elif size_last > size_latest:
            difference = last_data[latest_data.ne(last_data).any(axis=1)]
        else:
            difference = latest_data[last_data.ne(latest_data).any(axis=1)]
        
        filtered_data = update_index(difference)
        return filtered_data
    else:
        print("No update performed as index flag is not set to 0")
        return 0


In [58]:
#test index
if BUILD_INDEX_TEST:

    term ='machine'
    start_time = time.time() # measure time taken to search data     
    ps=PorterStemmer()

    test_word =ps.stem(term)
    
    if test_word in inv_index.keys():
        posting =inv_index[test_word]

        stop_time = time.time() # stop the timer
        time_taken = stop_time-start_time
        print("\nTime taken to search term in Index :",str(np.round(time_taken,4))+" sec") 
        #test index
    #     print(posting)
        if len(posting)!= 0:
            print("Searched term: ",term)
            print("Total matching documents (",len(posting),") are found::")
            for position,_ in zip(range(len(posting)),range(3)): 
                doc_id = posting[position][0]
                print("\nDoc_Id: ",doc_id,end=' : ')
                print("\nDoc_Link: ",list(display_data[display_data['Document_Id'] == doc_id]['Title_Link']))
                print("Title : ", list(display_data[display_data['Document_Id'] == doc_id]['Title']))
                print("Profile Name : ", list(display_data[display_data['Document_Id'] == doc_id]['Profile_Name']))
                print("Authors Name : ", list(display_data[display_data['Document_Id'] == doc_id]['Co_Authors']))
                print("No.Of Citations : ", list(display_data[display_data['Document_Id'] == doc_id]['No_Of_Citations']))
                
    else:
        print("Searched Term: ",test_word)
        print("No Match found in the index..")
        

# Query Processor engine

In [59]:
#calculate ranking
def query_doc_score(term):
    document_id = []
    document_score = []
    df=pd.DataFrame(columns=['Document_Id','Document_Score'],index=None)
    if term in inv_index.keys():
        posting  = inv_index[term]
        for doc_id in posting:
            tf = calc_frequency(term,doc_id[0])
            idf = calc_idf_frequency(term)
            score_idf = calc_tf_idf_frequency(tf,idf)
            document_id.append(doc_id[0])
            document_score.append(score_idf)             

        df['Document_Id'],df['Document_Score']=document_id,document_score
        return df
    else:
        return df # empty data frame
    
    
#calculate ter frequency 
#TF(t,d) =  Log10 (1 + frequency of t in d)
def calc_frequency(term,document):
    global inv_index
    if term in inv_index.keys():
        for i in range(len(inv_index[term])):
            if inv_index[term][i][0] == document:
#                 print(term, inv_index[term][i][1])
                return ( round(math.log10(1+inv_index[term][i][1]),4))
    else:
        return -1
    
#calculate inverted document frequency ( idf)
#IDF(t) = log10 (N/(df(t))
def calc_idf_frequency(term):
    global inv_index
    if term in inv_index.keys():
        total_no_of_documents = max(inv_index.values())[-1][0] # find total no.of docuements

        document_freq = word_frequency_count(inv_index[term])[0] # find document containing terms
#         print(total_no_of_documents, document_freq)
        return  math.log(total_no_of_documents/document_freq)
    else:
        return -1

#calcuate score tf.tdf 
def calc_tf_idf_frequency(tf, idf):
#     print(tf, idf,tf*idf)
    return round(tf*idf,4)

#returns word frequency and and no. of documents 
def word_frequency_count(posting):
    word_freq =0
    for inv_index in range(len(posting)):
        word_freq = word_freq+posting[inv_index][1]
        
    return(len(posting),word_freq)


#update document score for all the terms in a query
def update_document_score_query(term_doc_score):
    global document_score_query 
    
    for doc_id,doc_score in zip(term_doc_score['Document_Id'],term_doc_score['Document_Score']) :
#         print("Doc_id: ",doc_id,doc_score)  
        if doc_id not in document_score_query.keys():
            document_score_query[doc_id] = doc_score
        else:
#             print("Old score: ",  document_score_query[doc_id])
            document_score_query[doc_id] = document_score_query[doc_id]+doc_score
#             print("New score: ",  document_score_query[doc_id])
 
# global query score dictionary
document_score_query = {"doc_id":[],"doc_score":[]}

def query_processor_engine(query):
    # initialize STEM and stopwords
    global inv_index
    not_found = []
    processed = False
    ps=PorterStemmer()
    exlcude_puncuation = [';', '.', ',', '!', ':', '-','_']
    #set stopwords for english language
    stop_words = stopwords.words('english')
    
    # tokenize
    token = word_tokenize(query)
    term_freq_table = {}
    # remove sropwords
    for word in token:
        if word not in stop_words and word not in exlcude_puncuation:        
        # Stem the word
            stem_word = ps.stem(word)
#             print(stem_word)
#             print("Term: ", stem_word)
            print(stem_word, end=' ')
            if stem_word in inv_index.keys():
                processed = True
                posting = inv_index[stem_word]
                #print("Index: ", posting)
                if stem_word not in term_freq_table.keys():
                    term_freq_table[stem_word] = word_frequency_count(posting)[1]
                    term_doc_score = query_doc_score(stem_word)
#                     print(stem_word, end='+')
                    update_document_score_query(term_doc_score)

    #             print(stem_word," Occurred in ", count_word_frequency(posting)[0], \
    #                   " documents", count_word_frequency(posting)[1] ," times\n")
            else:
                not_found.append(stem_word)
    if len(not_found)!=0:
        print("\nNo match found for : ", not_found)
    return processed,not_found
    

In [60]:
#Test Query processor engine
if TEST_QUERY:

    query = "ramanathan perumal"
    document_score_query.clear()

    start_time = time.time() # measure time taken to search data   
    print("Input Query ->",query)
    print("Processing Query :", end=':')
    result,not_found = query_processor_engine(query)
    stop_time = time.time() # stop the timer
    time_taken = stop_time-start_time
    print("\n\nFound ",len(document_score_query), "results", "in",str(np.round(time_taken,4))+" sec" )
    results = {}
    if result == True:
    #     print(document_score_query)
        for i, key in zip(range(len(document_score_query)),sorted(document_score_query, key=document_score_query.get,reverse=True)):
            print("\nDoc_Id: ",key, "Relevance Score: ",round(document_score_query[key],4))
            print("Title: ",display_data.iloc[key-1]['Title'])
            print("Title Link: ", display_data.iloc[key-1]['Title_Link'])
            print("Profile Name: ",display_data.iloc[key-1]['Profile_Name'] )        
            print("Co Authors: ",display_data.iloc[key-1]['Co_Authors'])
            results[key] = [round(document_score_query[key],4),\
                            display_data.iloc[key-1]['Title'],\
                            display_data.iloc[key-1]['Title_Link'],\
                            display_data.iloc[key-1]['Profile_Name'],\
                            display_data.iloc[key-1]['Co_Authors'],\
                            display_data.iloc[key-1]['No_Of_Citations']]       
            break
    else:
        print("No matching data found...")
    

# User interface

In [64]:
from flask import Flask, render_template,request
import pickle

In [65]:
#load subject classification  pre-requisite
def subject_classification_model(query):
    #load the subject classification model
    model_name = 'NB_Model.sav'
    subject_classify_model = pickle.load(open(model_name, 'rb')) 
    #Load training data to create vectorspace
    x_train = pickle.load(open("NB_train_data.npy",'rb'))
#     print(x_train.shape,type(x_train))
    # Form tf-idf vector
    vectorizer = TfidfVectorizer()
    x_train_vector = vectorizer.fit_transform(x_train)
    
    stop_words = stopwords.words('english')
    ps = PorterStemmer()
    
    #convert into tokens, remove stop words and stem the tokens
    tokenizer = RegexpTokenizer('[A-Za-z]\w+')
    test_input = tokenizer.tokenize(str(query))
    
    test_input = [ps.stem(token) for token in test_input if token not in stop_words]      
    test_input =  [' '.join(map(str,test_input))]
    
    print("Inside function:", test_input)
    test_input = np.array(test_input)
    test_vector = vectorizer.transform(test_input)        

    return subject_classify_model, test_vector

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def vertical_Engine_IR():
    query = []
    errors = []
    results={}
    text_boot = False
    status = []
    not_found = []
    search = True
    subject=[]
    input_mapping = {0:"business",
                       1:"entertainment",
                       2:"food",
                       3:"graphics",
                       4:"historical",
                       5:"medical",
                       6:"politics",
                       7:"space",
                       8:"sport",
                       9:"technologie",
                    } 
    

    #if user has entered the search query
    if request.method == 'POST':
        try:
            #get user query
            query = request.form['text_Name']
            
            if request.form['search_query']=='Searching CU Authors':
                print("Searching CU Authors")
                search =True
            elif request.form['search_query']=='Searching Subject Classification':
                print("Searching subject classification")
                search=False
            
            document_score_query.clear()
            results.clear()              
            status.clear()
            not_found.clear()
            subject=[]    
            print(query)
            
        except:
            errors.append("Query Error.")
            return render_template('index.html',errors = errors)
        
        if query and search==True:
            
            text_boot = True
            start_time = time.time() # measure time taken to search data 
            print("input query ::",query)
            search,not_found = query_processor_engine(str(query))
            stop_time = time.time() # stop the timer
            time_taken = stop_time-start_time
#           print("\n\nFound ",len(document_score_query), "results", "in",str(np.round(time_taken,4))+" sec" )
            status.append(len(document_score_query)) # no. of documents found
            status.append(np.round(time_taken,4)) # time taken to process query
#             print(stats)
            if search == True:
                for i, key in zip(range(len(document_score_query)),sorted(document_score_query, key=document_score_query.get,reverse=True)):
                    results[key] = [round(document_score_query[key],4),\
                                    display_data.iloc[key-1]['Title'],\
                                    display_data.iloc[key-1]['Title_Link'],\
                                    display_data.iloc[key-1]['Profile_Name'],\
                                    display_data.iloc[key-1]['Co_Authors'],\
                                    display_data.iloc[key-1]['No_Of_Citations'] ]
            else:
                results = ''
        else:
            results = ''  # set null for search result
            print("Query : ",query)
            subject_classify_model,test_vector = subject_classification_model(query)
            
            result = subject_classify_model.predict(test_vector)
            prob = subject_classify_model.predict_proba(test_vector)
            print(prob*100)
            subject.append(input_mapping[result[0]])
            subject.append(round(prob[0][np.argmax(prob)]*100,2))
    return render_template('index.html',errors = errors, results=results,\
                           text_boot=text_boot,status=status,not_found=not_found,\
                          query = query,subject=subject)

print(__name__)
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

__main__
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Aug/2021 13:10:03] "GET / HTTP/1.1" 200 -


Searching subject classification
Football
Query :  Football
Inside function: ['footbal']


127.0.0.1 - - [02/Aug/2021 13:10:08] "POST / HTTP/1.1" 200 -


[[ 9.00044499 11.14145388  9.10197508  9.23776092  8.49394886  9.18993396
  10.19600926  8.96773925 16.25050129  8.42023252]]
Searching subject classification
virus
Query :  virus
Inside function: ['viru']


127.0.0.1 - - [02/Aug/2021 13:10:23] "POST / HTTP/1.1" 200 -


[[ 9.5787941   8.61525792  8.26934767  8.39271213  7.71694226  8.73301252
   9.26330216  8.14739142 10.05313259 21.23010721]]
Searching subject classification
trade
Query :  trade
Inside function: ['trade']


127.0.0.1 - - [02/Aug/2021 13:11:06] "POST / HTTP/1.1" 200 -


[[26.57446351  8.96423318  6.48364957  6.72107151 11.62860098  6.65960338
  11.0581723   6.5176093   7.28497398  8.10762228]]
